# Dataplex Data Quality and BigQuery ETL Demo

This notebook demonstrates a common data engineering workflow on Google Cloud. It covers:

1.  **Infrastructure Setup**: Creating a Google Cloud Storage (GCS) bucket and a BigQuery dataset.
2.  **Data Ingestion**: Loading raw data into BigQuery from both in-memory sources (managed tables) and GCS (external tables).
3.  **Data Quality**: Defining and creating Dataplex Data Quality scans to profile and validate the raw data.
4.  **ETL/ELT**: Transforming the raw data into a simple star-schema dimensional model.
5.  **Cleanup**: Providing a utility to tear down all created resources.

The data used is a mock dataset representing customers, products, and sales orders.

## 0. Setup and Configuration

First, we'll import the necessary libraries, configure our project variables, and initialize the Google Cloud clients.

### Authentication (If running locally)
If you are running this notebook from a local environment (not a GCP Vertex AI Notebook), you will need to authenticate. Uncomment and run the following cell.

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

In [ ]:
import csv
import io
import random
import time

from google.cloud import bigquery, storage, dataplex_v1
from google.api_core import exceptions
from google.api_core import exceptions as dataplex_exceptions

# ---  Configuration  ---
# TODO: Update these values with your project details
PROJECT_ID = "bq-sme-governance-build" # Your Google Cloud project ID
LOCATION = "us-central1"             # The region for your resources
DATASET_ID = "sme_raw_layer"         # The BigQuery dataset to create

BUCKET_NAME = f"{PROJECT_ID}-lab-data-source"
# ---------------------------

# Initialize clients
bq_client = bigquery.Client(project=PROJECT_ID)
storage_client = storage.Client(project=PROJECT_ID)
dataplex_client = dataplex_v1.DataScanServiceClient()

# Global list to track created scans for later execution and cleanup
CREATED_SCAN_NAMES = []

## 1. Helper Functions

Here we define all the helper functions that will be used throughout the notebook. They are grouped by purpose: data generation, resource management, data loading, data quality, and cleanup.

### Data Generation Functions
These functions generate mock data for our tables. Note that some data is intentionally malformed to demonstrate the effectiveness of the data quality scans.

In [ ]:
def get_mock_customers():
    """Generates mock data for the 'customers' table."""
    return [
        {"customer_id": "C1001", "first_name": "Alice", "last_name": "Smith", "email": "alice@example.com", "join_date": "2023-01-15"},
        {"customer_id": "C1002", "first_name": "Bob", "last_name": "Johnson", "email": "bob@example.com", "join_date": "2023-02-10"},
        {"customer_id": "C1003", "first_name": "Charlie", "last_name": "Brown", "email": "charlie@example.com", "join_date": "2023-03-05"},
        {"customer_id": "C1004", "first_name": "David", "last_name": "Lee", "email": "david@example.com", "join_date": "2023-04-20"},
        {"customer_id": "C1005", "first_name": "Eve", "last_name": "Davis", "email": "eve@example.com", "join_date": "2023-05-15"},
    ]

def get_mock_products():
    """Generates mock data for the 'products' table."""
    return [
        {"product_id": "P2001", "product_name": "Laptop", "category": "Electronics", "unit_price": 1200.00},
        {"product_id": "P2002", "product_name": "Mouse", "category": "Electronics", "unit_price": 25.50},
        {"product_id": "P2003", "product_name": "Coffee Mug", "category": "Homeware", "unit_price": 15.00},
        {"product_id": "P2004", "product_name": "Notebook", "category": "Stationery", "unit_price": 5.75},
        {"product_id": "P9999", "product_name": "Test Item", "category": "", "unit_price": -1.00}, # Intentional bad data
    ]

def get_mock_orders():
    """Generates mock data for 'orders' as a list of dicts."""
    return [
        {"order_id": "E101", "customer_id": "C1001", "order_date": "2024-05-01", "status": "Shipped"},
        {"order_id": "E102", "customer_id": "C1002", "order_date": "2024-05-03", "status": "Processing"},
        {"order_id": "E103", "customer_id": "C1001", "order_date": "2024-05-04", "status": "Shipped"},
        {"order_id": "E104", "customer_id": "C1003", "order_date": "2024-05-05", "status": "Delivered"},
        {"order_id": "E105", "customer_id": "C1004", "order_date": "2024-05-06", "status": "Shipped"},
        {"order_id": "E106", "customer_id": "C9999", "order_date": "2024-05-07", "status": "Pending"}, # Intentional bad data (FK violation)
    ]

def get_mock_order_items_csv():
    """Generates mock data for 'order_items' as a CSV string."""
    data = [
        ["item_id", "order_id", "product_id", "quantity"],
        ["OI301", "E101", "P2001", 1],
        ["OI302", "E101", "P2002", 1],
        ["OI303", "E102", "P2003", 2],
        ["OI304", "E103", "P2004", 5],
        ["OI305", "E104", "P2001", 1],
        ["OI306", "E105", "P2003", 1],
        ["OI307", "E106", "P9999", 99],
    ]
    output = io.StringIO()
    writer = csv.writer(output)
    writer.writerows(data)
    return output.getvalue()

### Resource Functions

In [ ]:
def ensure_gcs_bucket_exists():
    """Checks for GCS bucket and creates it if not found."""
    print(f"Checking for GCS bucket: {BUCKET_NAME}...")
    try:
        bucket = storage_client.get_bucket(BUCKET_NAME)
        print("...bucket already exists.")
    except exceptions.NotFound:
        print("...bucket not found, creating new bucket.")
        bucket = storage_client.create_bucket(BUCKET_NAME, location=LOCATION)
        print(f"...created bucket {bucket.name} in {bucket.location}")
    return bucket

def ensure_bq_dataset_exists():
    """Checks for BQ dataset and creates it if not found."""
    dataset_ref = bq_client.dataset(DATASET_ID)
    print(f"Checking for BigQuery dataset: {DATASET_ID}...")
    try:
        bq_client.get_dataset(dataset_ref)
        print("...dataset already exists.")
    except exceptions.NotFound:
        print("...dataset not found, creating new dataset.")
        dataset = bigquery.Dataset(dataset_ref)
        dataset.location = LOCATION
        bq_client.create_dataset(dataset, timeout=30)
        print("...created dataset.")

### BigQuery Data Loading Functions

In [ ]:
def upload_to_gcs(bucket, blob_name, data_string):
    """Uploads a string as a file to GCS."""
    print(f"Uploading {blob_name} to GCS bucket {bucket.name}...")
    blob = bucket.blob(blob_name)
    blob.upload_from_string(data_string, content_type="text/csv")
    print("...upload complete.")
    return f"gs://{bucket.name}/{blob_name}"

def load_table_from_memory(table_id, data, schema):
    """Loads data from a list of dicts into a new BQ table."""
    full_table_id = f"{PROJECT_ID}.{DATASET_ID}.{table_id}"
    print(f"Starting load job for table: {full_table_id}...")
    job_config = bigquery.LoadJobConfig(
        schema=schema,
        write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE,
    )
    try:
        load_job = bq_client.load_table_from_json(
            data, full_table_id, job_config=job_config
        )
        load_job.result()
        print("...load job finished.")
    except Exception as e:
        print(f"Error loading table {full_table_id}: {e}")

def create_external_table(table_id, schema, gcs_uri):
    """Creates a new BQ external table pointing to a GCS file."""
    full_table_id = f"{PROJECT_ID}.{DATASET_ID}.{table_id}"
    print(f"Creating external table: {full_table_id}...")
    external_config = bigquery.ExternalConfig("CSV")
    external_config.source_uris = [gcs_uri]
    external_config.schema = schema
    external_config.csv_options.skip_leading_rows = 1
    table = bigquery.Table(full_table_id)
    table.external_data_configuration = external_config
    try:
        bq_client.delete_table(full_table_id, not_found_ok=True)
        bq_client.create_table(table)
        print("...external table created.")
    except Exception as e:
        print(f"Error creating external table {full_table_id}: {e}")

### Dataplex Data Quality Functions

In [ ]:
def get_customers_dq_rules():
    """Returns a list of appropriate DQ rules for the 'customers' table."""
    return [
        {"column": "customer_id", "non_null_expectation": {}, "dimension": "VALIDITY", "description": "Customer ID must not be empty."},
        {"column": "customer_id", "uniqueness_expectation": {}, "dimension": "UNIQUENESS", "description": "Each Customer ID must be unique."},
        {"column": "email", "regex_expectation": {"regex": r"^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$"}, "dimension": "VALIDITY", "description": "Email must be in a valid format."},
    ]

def get_products_dq_rules():
    """Returns a list of appropriate DQ rules for the 'products' table."""
    return [
        {"column": "product_id", "non_null_expectation": {}, "dimension": "VALIDITY", "description": "Product ID must not be empty."},
        {"column": "unit_price", "range_expectation": {"min_value": "0.01"}, "dimension": "VALIDITY", "description": "Unit price must be a positive value."},
        {"column": "category", "set_expectation": {"values": ["Electronics", "Homeware", "Stationery"]}, "dimension": "VALIDITY", "description": "Category must be one of the allowed values."},
    ]

def get_orders_dq_rules():
    """Returns a list of appropriate DQ rules for the 'orders' table."""
    customers_table_fqn = f"`{PROJECT_ID}.{DATASET_ID}.customers`"
    return [
        {"column": "order_id", "uniqueness_expectation": {}, "dimension": "UNIQUENESS", "description": "Each Order ID must be unique."},
        {"row_condition_expectation": {"sql_expression": f"customer_id IS NULL OR customer_id IN (SELECT customer_id FROM {customers_table_fqn})"}, "dimension": "CONSISTENCY", "description": "Customer ID must exist in the customers table."},
    ]

def get_order_items_dq_rules():
    """Returns a list of appropriate DQ rules for the 'order_items' table."""
    orders_table_fqn = f"`{PROJECT_ID}.{DATASET_ID}.orders`"
    products_table_fqn = f"`{PROJECT_ID}.{DATASET_ID}.products`"
    return [
        {"column": "quantity", "range_expectation": {"min_value": "1"}, "dimension": "VALIDITY", "description": "Quantity must be at least 1."},
        {"row_condition_expectation": {"sql_expression": f"order_id IS NULL OR order_id IN (SELECT order_id FROM {orders_table_fqn})"}, "dimension": "CONSISTENCY", "description": "Order ID must exist in the orders table."},
        {"row_condition_expectation": {"sql_expression": f"product_id IS NULL OR product_id IN (SELECT product_id FROM {products_table_fqn})"}, "dimension": "CONSISTENCY", "description": "Product ID must exist in the products table."},
    ]

def create_data_quality_scan(project_id, table_name, rules):
    """Creates a Dataplex DQ scan for a given BigQuery table."""
    clean_table_name = table_name.replace('_', '-')
    scan_id = f"dq-{clean_table_name}-{random.randint(1000, 9999)}"
    table_resource_string = f"//bigquery.googleapis.com/projects/{project_id}/datasets/{DATASET_ID}/tables/{table_name}"
    
    print(f"\nCreating Dataplex DQ scan '{scan_id}' for table: {table_name}...")

    try:
        parent = f"projects/{project_id}/locations/{LOCATION}"

        data_scan = dataplex_v1.types.DataScan(
            data={"resource": table_resource_string},
            data_quality_spec={"rules": rules},
            execution_spec={"trigger": {"on_demand": {}}},
        )

        request = dataplex_v1.CreateDataScanRequest(
            parent=parent,
            data_scan=data_scan,
            data_scan_id=scan_id,
        )

        operation = dataplex_client.create_data_scan(request=request)
        result = operation.result()

        print(f"...Successfully created DQ scan: {result.name}")
        CREATED_SCAN_NAMES.append(result.name) # Store the full name for running/cleanup
        
    except Exception as e:
        print(f"!!! An error occurred creating the DQ scan for {table_name}: {e}")

def run_data_quality_scans():
    """Runs all the Dataplex DQ scans created by this script."""
    if not CREATED_SCAN_NAMES:
        print("...no scans were created, skipping.")
        return

    for scan_name in CREATED_SCAN_NAMES:
        print(f"...triggering scan: {scan_name.split('/')[-1]}")
        try:
            request = dataplex_v1.RunDataScanRequest(name=scan_name)
            response = dataplex_client.run_data_scan(request=request)
            print(f"  -> Scan run initiated. Job name: {response.job.name}")
        except Exception as e:
            print(f"!!! Error running scan {scan_name}: {e}")
    print("\nScans are running in the background. It may take a few minutes for results to appear in the Cloud Console.")

### Data Transformation (ETL) Functions

In [ ]:
def create_dimensional_tables():
    """Creates the dimension and fact tables from the base tables."""
    # dim_customers
    print("Creating dim_customers...")
    sql = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.dim_customers` AS
    SELECT
      customer_id AS customer_key,
      first_name,
      last_name,
      email,
      join_date
    FROM
      `{PROJECT_ID}.{DATASET_ID}.customers`
    """
    bq_client.query(sql).result()

    # dim_products
    print("Creating dim_products...")
    sql = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.dim_products` AS
    SELECT
      product_id AS product_key,
      product_name,
      category,
      unit_price
    FROM
      `{PROJECT_ID}.{DATASET_ID}.products`
    """
    bq_client.query(sql).result()

    # dim_date
    print("Creating dim_date...")
    sql = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.dim_date` AS
    SELECT
      order_date AS date_key,
      EXTRACT(YEAR FROM order_date) AS year,
      EXTRACT(MONTH FROM order_date) AS month,
      EXTRACT(DAY FROM order_date) AS day,
      EXTRACT(DAYOFWEEK FROM order_date) AS day_of_week
    FROM (
      SELECT DISTINCT order_date FROM `{PROJECT_ID}.{DATASET_ID}.orders`
    )
    """
    bq_client.query(sql).result()

    # fct_sales
    print("Creating fct_sales...")
    sql = f"""
    CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET_ID}.fct_sales` AS
    SELECT
      oi.item_id AS sales_key,
      o.order_id,
      o.customer_id AS customer_key,
      oi.product_id AS product_key,
      o.order_date AS order_date_key,
      oi.quantity,
      p.unit_price,
      oi.quantity * p.unit_price AS total_price
    FROM
      `{PROJECT_ID}.{DATASET_ID}.order_items` AS oi
    JOIN
      `{PROJECT_ID}.{DATASET_ID}.orders` AS o ON oi.order_id = o.order_id
    JOIN
      `{PROJECT_ID}.{DATASET_ID}.products` AS p ON oi.product_id = p.product_id
    """
    bq_client.query(sql).result()

    print("...dimensional tables created.")

### Cleanup Functions

In [ ]:
def cleanup_dataplex_scans():
    """Deletes all Dataplex scans created by this script."""
    print("\nAttempting to delete Dataplex Data Quality Scans...")
    if not CREATED_SCAN_NAMES:
        print("...no scans were created, skipping.")
        return

    deleted_count = 0
    for scan_name in CREATED_SCAN_NAMES:
        print(f"...deleting scan: {scan_name.split('/')[-1]}")
        try:
            request = dataplex_v1.DeleteDataScanRequest(name=scan_name)
            operation = dataplex_client.delete_data_scan(request=request)
            operation.result(timeout=120)
            print("  -> Scan deleted successfully.")
            deleted_count += 1
        except dataplex_exceptions.NotFound:
            print("  -> Scan not found, may have already been deleted.")
        except Exception as e:
            print(f"!!! Error deleting scan {scan_name}: {e}")
    print(f"...deleted {deleted_count} Dataplex scans.")

def cleanup_resources():
    """Deletes all created resources for a clean teardown."""
    print("\n--- STARTING RESOURCE CLEANUP ---")
    cleanup_dataplex_scans()
    print(f"\nAttempting to delete GCS Bucket: {BUCKET_NAME}...")
    try:
        bucket = storage_client.bucket(BUCKET_NAME)
        bucket.delete(force=True)
        print(f"...bucket {BUCKET_NAME} deleted successfully.")
    except exceptions.NotFound:
        print(f"...bucket {BUCKET_NAME} not found, skipping.")
    except Exception as e:
        print(f"Error deleting bucket {BUCKET_NAME}: {e}")

    print(f"\nAttempting to delete BigQuery Dataset: {DATASET_ID}...")
    try:
        bq_client.delete_dataset(DATASET_ID, delete_contents=True, not_found_ok=True)
        print(f"...dataset {DATASET_ID} deleted successfully.")
    except Exception as e:
        print(f"Error deleting dataset {DATASET_ID}: {e}")
    print("\n--- CLEANUP COMPLETE ---")

---

## Main Execution Workflow

Now we will execute the helper functions in a logical sequence. You can run the cells one by one to see the process unfold.

### Step 1: Create GCS and BigQuery Infrastructure

In [ ]:
print(f"Starting data setup for project: {PROJECT_ID}\n")
try:
    bucket = ensure_gcs_bucket_exists()
    ensure_bq_dataset_exists()
except Exception as e:
    print(f"Failed to create cloud resources: {e}\nCheck permissions and config.")

### Step 2: Load Internal (Managed) Tables
We load `customers`, `products`, and `orders` data directly from memory into standard BigQuery tables.

In [ ]:
print("--- Handling Internal Tables (Load Jobs) ---")
customers_schema = [bigquery.SchemaField("customer_id", "STRING"), bigquery.SchemaField("first_name", "STRING"), bigquery.SchemaField("last_name", "STRING"), bigquery.SchemaField("email", "STRING"), bigquery.SchemaField("join_date", "DATE")]
load_table_from_memory("customers", get_mock_customers(), customers_schema)

products_schema = [bigquery.SchemaField("product_id", "STRING"), bigquery.SchemaField("product_name", "STRING"), bigquery.SchemaField("category", "STRING"), bigquery.SchemaField("unit_price", "FLOAT64")]
load_table_from_memory("products", get_mock_products(), products_schema)

orders_schema = [bigquery.SchemaField("order_id", "STRING"), bigquery.SchemaField("customer_id", "STRING"), bigquery.SchemaField("order_date", "DATE"), bigquery.SchemaField("status", "STRING")]
load_table_from_memory("orders", get_mock_orders(), orders_schema)

### Step 3: Load External Table
For the `order_items` table, we first upload the data as a CSV to our GCS bucket and then create an external BigQuery table that reads directly from that GCS file.

In [ ]:
print("\n--- Handling External Tables (GCS) ---")
order_items_gcs_uri = upload_to_gcs(bucket, "raw/order_items/order_items.csv", get_mock_order_items_csv())
order_items_schema = [bigquery.SchemaField("item_id", "STRING"), bigquery.SchemaField("order_id", "STRING"), bigquery.SchemaField("product_id", "STRING"), bigquery.SchemaField("quantity", "INTEGER")]
create_external_table("order_items", order_items_schema, order_items_gcs_uri)

### Step 4: Create and Run Dataplex Data Quality Scans

Now that our raw data tables exist, we can define and create on-demand Dataplex scans to check for issues. After creating the scans, we trigger them to run.

In [ ]:
print("\n--- Creating Dataplex Data Quality Scans ---")
create_data_quality_scan(PROJECT_ID, "customers", get_customers_dq_rules())
create_data_quality_scan(PROJECT_ID, "products", get_products_dq_rules())
create_data_quality_scan(PROJECT_ID, "orders", get_orders_dq_rules())
create_data_quality_scan(PROJECT_ID, "order_items", get_order_items_dq_rules())

# Wait a moment for scan creation to propagate before running
print("\nWaiting 5 seconds before triggering scans...")
time.sleep(5)

print("\n--- Running Dataplex Data Quality Scans ---")
run_data_quality_scans()

### Step 5: Create Dimensional Model
Finally, we run SQL queries to transform our raw tables into a star schema consisting of dimension tables (`dim_customers`, `dim_products`, `dim_date`) and a fact table (`fct_sales`).

In [ ]:
print("\n--- Creating Dimensional Tables ---")
create_dimensional_tables()

### Setup Complete

In [ ]:
print("\n--- Setup Complete ---")
print(f"Project: {PROJECT_ID}")
print(f"Dataset: {DATASET_ID}")
print("Resources created:")
print(" - Tables: customers, products, orders, order_items, dim_customers, dim_products, dim_date, fct_sales")
print(" - Dataplex DQ Scans for all raw tables.")
print(" - GCS Bucket for external data.")

---

## 6. Resource Cleanup

**Warning:** The following cell is destructive. It will delete all the resources created by this notebook, including the GCS bucket (and its contents), the BigQuery dataset (and all its tables), and the Dataplex data scans.

Run this cell only when you are finished and want to clean up your project.

In [ ]:
# To automatically delete all created resources, uncomment and run the line below.

#cleanup_resources()